### Challenge 17: The CBC padding oracle

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

In [1]:
# %% Initialize

import cryptopals as cp
import numpy as np
import base64
from numpy.random import randint

# ---------UNKNOWN PARAMETERS ------------
unknown_key = bytes(list(randint(0, 256, 16)))
unknown_IV = bytes(list(randint(0, 256, 16)))
# ---------END UNKNOWN PARAMETERS---------

block_size = 16

<div class="alert alert-block alert-info">
    
The CBC padding oracle

This is the best-known attack on modern block-cipher cryptography.

Combine your padding code and your CBC code to write two functions.
    
</div>

<div class="alert alert-block alert-info">
    
The first function should select at random one of the following 10 strings:
    
```
MDAwMDAwTm93IHRoYXQgdGhlIHBhcnR5IGlzIGp1bXBpbmc=
MDAwMDAxV2l0aCB0aGUgYmFzcyBraWNrZWQgaW4gYW5kIHRoZSBWZWdhJ3MgYXJlIHB1bXBpbic=
MDAwMDAyUXVpY2sgdG8gdGhlIHBvaW50LCB0byB0aGUgcG9pbnQsIG5vIGZha2luZw==
MDAwMDAzQ29va2luZyBNQydzIGxpa2UgYSBwb3VuZCBvZiBiYWNvbg==
MDAwMDA0QnVybmluZyAnZW0sIGlmIHlvdSBhaW4ndCBxdWljayBhbmQgbmltYmxl
MDAwMDA1SSBnbyBjcmF6eSB3aGVuIEkgaGVhciBhIGN5bWJhbA==
MDAwMDA2QW5kIGEgaGlnaCBoYXQgd2l0aCBhIHNvdXBlZCB1cCB0ZW1wbw==
MDAwMDA3SSdtIG9uIGEgcm9sbCwgaXQncyB0aW1lIHRvIGdvIHNvbG8=
MDAwMDA4b2xsaW4nIGluIG15IGZpdmUgcG9pbnQgb2g=
MDAwMDA5aXRoIG15IHJhZy10b3AgZG93biBzbyBteSBoYWlyIGNhbiBibG93
```
<br>
... generate a random AES key (which it should save for all future encryptions), pad the string out to the 16-byte AES block size and CBC-encrypt it under that key, providing the caller the ciphertext and IV.
    
</div>

In [2]:
def Challenge17Part1(key, IV):

    rndStrings = [b'MDAwMDAwTm93IHRoYXQgdGhlIHBhcnR5IGlzIGp1bXBpbmc=',
                  b'MDAwMDAxV2l0aCB0aGUgYmFzcyBraWNrZWQgaW4gYW5kIHRoZSBWZWdhJ\
                      3MgYXJlIHB1bXBpbic=',
                  b'MDAwMDAyUXVpY2sgdG8gdGhlIHBvaW50LCB0byB0aGUgcG9pbnQsIG5vI\
                      GZha2luZw==',
                  b'MDAwMDAzQ29va2luZyBNQydzIGxpa2UgYSBwb3VuZCBvZiBiYWNvbg==',
                  b'MDAwMDA0QnVybmluZyAnZW0sIGlmIHlvdSBhaW4ndCBxdWljayBhbmQgb\
                      mltYmxl',
                  b'MDAwMDA1SSBnbyBjcmF6eSB3aGVuIEkgaGVhciBhIGN5bWJhbA==',
                  b'MDAwMDA2QW5kIGEgaGlnaCBoYXQgd2l0aCBhIHNvdXBlZCB1cCB0ZW1wb\
                      w==',
                  b'MDAwMDA3SSdtIG9uIGEgcm9sbCwgaXQncyB0aW1lIHRvIGdvIHNvbG8=',
                  b'MDAwMDA4b2xsaW4nIGluIG15IGZpdmUgcG9pbnQgb2g=',
                  b'MDAwMDA5aXRoIG15IHJhZy10b3AgZG93biBzbyBteSBoYWlyIGNhbiBib\
                      G93']

    strIdx = randint(0, 10)
    plaintext = cp.PKCS7_pad(base64.b64decode(rndStrings[strIdx]))
    ciphertext = cp.AESEncrypt(plaintext, key, 'CBC', IV)

    return(ciphertext)

<div class="alert alert-block alert-info">
    
The second function should consume the ciphertext produced by the first function, decrypt it, check its padding, and return true or false depending on whether the padding is valid.
</div>

In [3]:
def Challenge17Part2(ciphertext, key, IV):

    plaintext = cp.AESDecrypt(ciphertext, key, 'CBC', IV)
    try:
        cp.strip_PKCS7_pad(plaintext)
        return(True)
    except:
        return(False)

<div class="alert alert-block alert-info">
    
<div class="alert alert-block alert-warning">
    
### What you're doing here.   

This pair of functions approximates AES-CBC encryption as its deployed serverside in web applications; the second function models the server's consumption of an encrypted session token, as if it was a cookie.

</div>

It turns out that it's possible to decrypt the ciphertexts provided by the first function.

The decryption here depends on a side-channel leak by the decryption function. The leak is the error message that the padding is valid or not.

You can find 100 web pages on how this attack works, so I won't re-explain it. What I'll say is this:

The fundamental insight behind this attack is that the byte 01h is valid padding, and occur in 1/256 trials of "randomized" plaintexts produced by decrypting a tampered ciphertext.

>02h in isolation is not valid padding.

>02h 02h is valid padding, but is much less likely to occur randomly than 01h.

>03h 03h 03h is even less likely.

So you can assume that if you corrupt a decryption AND it had valid padding, you know what that padding byte is.

It is easy to get tripped up on the fact that CBC plaintexts are "padded". Padding oracles have nothing to do with the actual padding on a CBC plaintext. It's an attack that targets a specific bit of code that handles decryption. You can mount a padding oracle on any CBC block, whether it's padded or not.
    
</div>

---
## Implement the Attack

In [4]:
part_1_out = Challenge17Part1(unknown_key, unknown_IV)
part_2_out = Challenge17Part2(part_1_out, unknown_key, unknown_IV)
print(part_2_out)

True


In [5]:
# Let's call the random function 20 times & find each plaintext
for test_idx in range(20):
    
    part_1_out = Challenge17Part1(unknown_key, unknown_IV)
    part_2_out = Challenge17Part2(part_1_out, unknown_key, unknown_IV)
    
    PT = b'****************'  # Remember, we can't find the 1st block's Plaintext with this attack
    N_blocks = len(part_1_out)//16

    for blk_idx in range(0, N_blocks-2):

        ao_x_known = b''                # This is the value of the AES intermediate output that we're going after
        start_idx = 16*(blk_idx+1)
        stop_idx = start_idx + 16

        for ii in range(15, -1, -1):    # Work backwards, starting w/ the last byte in the block

            for jj in range(0, 256):    # Try every possible character until we find one that gives us valid padding!

                random_prefix = list(randint(0, 256, ii))  # Probably not necessary...
                ct_0_p = random_prefix + [jj]

                ct_1 = list(part_1_out[start_idx:stop_idx])

                # Take previous PT and make padding match needed pad length for
                # bytes that are already known.
                pad_length = (16 - ii)
                for kk in range(len(ao_x_known)):
                    # We're working on one byte at a time.  
                    ct_0_p = ct_0_p + [ao_x_known[kk] ^ pad_length]

                chosen_CT = bytes(ct_0_p + ct_1)

                if Challenge17Part2(chosen_CT, unknown_key,
                                       unknown_IV):

                    # Valid padding was found...save this guess.
                    this_iv_byte = bytes([jj ^ pad_length])
                    ao_x_known = this_iv_byte + ao_x_known
                    break

                if jj == 255:
                    print(f'FAILED AT ii={ii}, jj={jj}')

            # end for jj

        # end for ii
        PT += cp.bitwise_xor(part_1_out[start_idx - 16:stop_idx - 16], ao_x_known)
    # end for blk_idx

    print('\nDecrypted Plaintext Is:\n')
    print(cp.strip_PKCS7_pad(PT).decode())


Decrypted Plaintext Is:

****************he party is jumping

Decrypted Plaintext Is:

****************he party is jumping

Decrypted Plaintext Is:

****************-top down so my hair can blow

Decrypted Plaintext Is:

****************-top down so my hair can blow

Decrypted Plaintext Is:

****************m, if you ain't quick and nimble

Decrypted Plaintext Is:

****************he point, to the point, no faking

Decrypted Plaintext Is:

****************-top down so my hair can blow

Decrypted Plaintext Is:

****************'s like a pound of bacon

Decrypted Plaintext Is:

****************oll, it's time to go solo

Decrypted Plaintext Is:

**************** when I hear a cymbal

Decrypted Plaintext Is:

****************oll, it's time to go solo

Decrypted Plaintext Is:

****************ass kicked in and the Vega's are pumpin'

Decrypted Plaintext Is:

****************my five point oh

Decrypted Plaintext Is:

****************he point, to the point, no faking

Decrypted Plaintext Is:

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)